In [1]:
!git clone https://github.com/HaiDang2001VN/tkgc-art.git src
!git clone https://github.com/stmrdus/TKGC-Benchmark-Datasets.git dataset
%mv dataset/Unified_Datasets src/data
%cd src/v5
%pip install -r requirements.txt
%pip install -r requirements_geometric.txt
!g++ -std=c++23 -O3 -pthread -o prepare.o prepare.cpp
!g++ -std=c++23 -O3 -pthread -o preprocess.o preprocess.cpp

Cloning into 'src'...
remote: Enumerating objects: 1549, done.
remote: Counting objects: 100% (212/212), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 1549 (delta 150), reused 131 (delta 72), pack-reused 1337 (from 1)
Receiving objects: 100% (1549/1549), 915.61 KiB | 8.32 MiB/s, done.
Resolving deltas: 100% (1087/1087), done.
Cloning into 'dataset'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 315 (delta 12), reused 131 (delta 11), pack-reused 178 (from 1)
Receiving objects: 100% (315/315), 67.12 MiB | 9.34 MiB/s, done.
Resolving deltas: 100% (41/41), done.
Updating files: 100% (147/147), done.
/content/src/v5
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu128
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.5/828.5 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 31.9 MB/s eta 0:00:00
 

In [95]:
%cd /content/src/v5
!sh run.sh data config_score.json
!sh run.sh prepare config_score.json

/content/src/v5
Running data stage with config config_score.json
train pos: 100% 539286/539286 [00:02<00:00, 216356.90it/s]
valid pos: 100% 67538/67538 [00:00<00:00, 177006.77it/s]
test pos: 100% 63110/63110 [00:00<00:00, 186310.25it/s]
Collected neighbor information from 12435 nodes
Adding inverse edges for 669934 original edges...
Creating inverse edges: 100% 669934/669934 [00:01<00:00, 363559.34it/s]
Added 669934 inverse edges. Total relations: 48
Generating negatives for train split using 2 workers...
train neg: 100% 377599/377599 [02:11<00:00, 2872.74it/s]
Killed
Running prepare stage with config config_score.json
Starting parallel shortest path computation with 4 threads...
Processing CSV: ../data/wiki_edges.csv
Max hops: 6
Decay Factor: 0.2
Max Fanout: 100
Processing threads: 100% 4/4 [00:00<00:00, 466.88it/s]
Collected 0 shortest paths from all threads
Writing shortest paths to text file: ../data/wiki_paths.txt
Successfully saved 0 shortest paths to ../data/wiki_paths.txt


In [96]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from loader import PathDataModule
from tqdm import tqdm

# --- 1. Configuration and Data Loading ---
config_path = 'config_score.json'
config_data = json.load(open(config_path, 'r'))
print(config_data)

{'dataset': 'wiki', 'storage_dir': '../data/', 'embedding_config': './embedding_config.json', 'criteria': 'score', 'train_ratio': 0.3, 'hidden_dim': 256, 'max_hops': 6, 'num_neg': 100, 'num_threads': 4, 'decay_factor': 0.2, 'max_fanout': 100, 'beam_width': 20, 'max_epochs': 1, 'batch_size': 8, 'dim_feedforward': 256, 'nhead': 4, 'num_layers': 6, 'dropout': 0.1, 'store': 'model', 'save_text_embeddings': True, 'shallow': True, 'pre_scan': ['train'], 'adjust_no_neg_paths_samples': True, 'max_adjust': 5.0, 'positive_deviation': True, 'embedding': 'all', 'test_time': 0, 'num_ckpt': 2, 'scale_loss': True, 'chi2': False, 'lr': 0.0001, 'wandb_project': 'thesis-graph'}


In [97]:
tqdm.pandas()

In [98]:
# import os, requests, json, pprint

# cid  = os.getenv("CONTAINER_ID")          # ← set by Vast.ai inside every container
# key  = os.getenv("CONTAINER_API_KEY")     # ← scoped token for this one instance
# assert cid and key, "Not running on a Vast.ai container!"

# resp = requests.get(
#     f"https://console.vast.ai/api/v0/instances/{cid}/",
#     headers={"Authorization": f"Bearer {key}",
#              "accept": "application/json"},
#     timeout=10,
# )

# info = resp.json()
# print("Effective vCPUs:", info['instances']["cpu_cores_effective"])
# # print(info['instances'].keys())

In [99]:

# Initialize the DataModule and load the data
print("Setting up DataModule...")
dm = PathDataModule(config_path=config_path, batch_size=32)
dm.setup('fit')
print("Data loaded.")


Setting up DataModule...
Setting up data for stage: fit


FileNotFoundError: [Errno 2] No such file or directory: '../data/wiki_edges.csv'

In [ ]:

# --- 2. Data Extraction and Preparation ---
plot_data = []
split = 'test'  # You can change this to 'valid' or 'test'

print(f"Processing data for '{split}' split...")
edges_df = dm.data[split]
pos_paths = dm.pos_paths[split]
neg_paths = dm.neg_paths[split]


In [ ]:
print(len(pos_paths), len(neg_paths))

In [ ]:
edges_df.info()

In [ ]:
edges_df[edges_df['v_pos'] == edges_df['v']].info()

In [ ]:
edges_df[edges_df['v_pos'] == edges_df['v']].describe()

In [ ]:
edges_df.groupby(["u", "v_pos", "edge_type", "ts"]).count()

In [ ]:

for eid, row in tqdm(edges_df.iterrows(), total=len(edges_df), desc="Extracting paths"):
    eid_str = str(eid)
    label = row['label']
    u = row['u']
    v = row['v']
    ts = row['ts']
    v_pos = row['v_pos']
    edge_type = row['edge_type']
    edge_meta = {}

    # Process positive path
    if eid_str in pos_paths and pos_paths[eid_str].get('nodes'):
        pos_path_len = len(pos_paths[eid_str]['nodes'])
        edge_meta = {
            'path_length': pos_path_len,
            # 'path_type': 'positive',
            'label': label,
            'u': u, 'v': v, 'ts': ts, 'edge_type': edge_type,
            'v_pos': v_pos

        }
    else:
        edge_meta = {
            'path_length': 0,
            # 'path_type': 'positive',
            'label': label,
            'u': u, 'v': v, 'ts': ts, 'edge_type': edge_type,
            'v_pos': v_pos
        }

    # # Process negative paths
    # if eid_str in neg_paths:
    #     # for neg_path_interleaved in neg_paths[eid_str]:
    #     #     # As per loader.py, nodes are at even indices
    #     #     neg_path_len = len(neg_path_interleaved[::2])
    #     #     plot_data.append({
    #     #         'path_length': neg_path_len,
    #     #         'path_type': 'negative',
    #     #         'label': 'true_link' if label == 1 else 'false_link'
    #     #     })
    #     edge_meta["num_neg"] = len(neg_paths[eid_str])

    plot_data.append(edge_meta)

In [ ]:

plot_df = pd.DataFrame(plot_data)
print("Data prepared for plotting.")


In [ ]:
plot_df.info()

In [ ]:
edges_fp = os.path.join(config_data['storage_dir'], f"{config_data['dataset']}_edges.csv")
df = pd.read_csv(edges_fp, index_col='edge_id')
split_map = {str(idx): row['split'] for idx, row in tqdm(df.iterrows())}

In [ ]:
df.info()

In [ ]:
df[(df['v_pos'] == df['v']) & (df['split'] == 3)].info()

In [ ]:
df[(df['label'] == 1) & (df['split'] == 3)].info()

In [ ]:
df[(df['v_pos'] == df['u']) & (df['split'] == 3)].info()

In [ ]:
df.groupby(["u", "v_pos", "edge_type", "ts"]).count()

In [ ]:
df[df["split"] == 2].groupby(["u", "v_pos", "edge_type", "ts"]).count()

In [ ]:
# len(split_map.keys())

In [ ]:
# list(split_map.values())[-1]

In [ ]:
# split_code = {'pre': 0, 'train': 1, 'valid': 2, 'test': 3}

# print(f"Setting up data for split: {split}")

# pos_paths = {}
# with open(os.path.join(config_data['storage_dir'], f"{config_data['dataset']}_paths.txt")) as f:
#     n_str = f.readline()
#     n = int(n_str) if n_str and n_str.strip() else 0
#     for _ in tqdm(range(n)):
#         eid = f.readline().strip()
#         if not eid:
#             break
#         hops = int(f.readline())
#         nodes = [int(u) for u in f.readline().split()]
#         node_types = [int(t) for t in f.readline().split()]
#         edge_types_str = f.readline().strip().split()
#         edge_types = [int(et) for et in edge_types_str if et]

#         edge_timestamps_str = f.readline().strip().split()
#         edge_timestamps = [int(ts) for ts in edge_timestamps_str if ts]

#         if split_map.get(eid) == split_code[split]:
#             pos_paths[eid] = {
#                 "hops": hops,
#                 "nodes": nodes,
#                 "node_types": node_types,
#                 "edge_types": edge_types,
#                 "edge_timestamps": edge_timestamps
#             }


In [ ]:
# len(pos_paths.keys())

In [ ]:
plot_df

In [ ]:
pos_df = plot_df[plot_df["path_length"] > 0].copy()
pos_df

In [ ]:
temp = pos_df[pos_df["label"] == 1].groupby("path_length").count()
temp

In [ ]:
temp / temp.sum() * 100

In [ ]:
pos_df[pos_df["label"] == 0].groupby("path_length").count()

In [ ]:
plot_df[plot_df["path_length"] < 2]

In [ ]:
def calculate_metrics(group):
    """
    Calculates MRR and Hits@K for a group of predictions for a single query.
    The group contains one 'true_link' and multiple 'false_link' rows.
    A lower 'path_length' is considered a better score.
    """
    true_link = group[group['label'] == 1]
    if true_link.empty or (true_link['path_length'].min() == 0):
        min_neg = 0 if group.loc[group['label'] == 1, 'path_length'].min() > 0 else 1
        return pd.Series({
            'rank': min_neg-1, 'mrr': min_neg, 'hits@1': min_neg, 'hits@3': min_neg, 'hits@10': min_neg
        })

    # Lower path_length is better.
    true_path_length = true_link['path_length'].min()

    # Rank is 1 + number of negative samples with a better (smaller) or equal path length.
    # We use '<=' because if scores are tied, the true link does not get the best rank.
    rank = 1 + group[(group['label'] == 0) & (group['path_length'] <= true_path_length)].shape[0]

    mrr = 1.0 / rank
    hits_at_1 = 1.0 if rank <= 1 else 0.0
    hits_at_3 = 1.0 if rank <= 3 else 0.0
    hits_at_10 = 1.0 if rank <= 10 else 0.0

    return pd.Series({
        'rank': rank,
        'mrr': mrr,
        'hits@1': hits_at_1,
        'hits@3': hits_at_3,
        'hits@10': hits_at_10
    })


In [ ]:

# Group by the query identifiers.
# This assumes that for each (u, v, ts, edge_type) combination that forms a true link,
# there are corresponding negative samples that share these identifiers in some way
# that allows grouping. If negative samples have different u or v, the grouping key needs adjustment.
# Based on the request, we group by ('u', 'v', 'ts', 'edge_type').
grouped = plot_df.groupby(['u', "v_pos", "edge_type", "ts"])


In [ ]:

# Apply the function to each group and get the results.
metrics_df = grouped.progress_apply(calculate_metrics, include_groups=False).reset_index()


In [ ]:

print("Metrics per query:")
metrics_df


In [ ]:

print("\nAverage metrics across all queries:")
metrics = ['mrr', 'hits@1', 'hits@3', 'hits@10']
metrics_df[metrics].mean() * 100


In [ ]:
temp_metrics = metrics_df.groupby("ts")[metrics].mean().reset_index()
temp_metrics

In [ ]:
temp_metrics.plot.line(x="ts", y=metrics)

In [ ]:
metrics_df['rank'].plot.hist(bins=10)

In [ ]:
metrics_df.groupby('rank').count()

In [ ]:
metric_names = ["mrr", "hits@1", "hits@3", "hits@10"]
time_metrics = metrics_df.groupby("ts")[metric_names].mean().reset_index()
time_metrics

In [ ]:
time_metrics.plot.line(x="ts", y=metric_names)

In [ ]:
type_metrics = metrics_df.groupby("edge_type")[metric_names].mean().reset_index()
type_metrics

In [ ]:
ax = type_metrics["hits@1"].plot.hist(bins=20)

In [ ]:
results = pd.read_csv("../data/transe/icews14/thesis-graph/iostv0db/train_val_0_scores.csv")
results

In [ ]:
len(results)

In [ ]:
len(results[results['label'] == 0])

In [ ]:
len(results[results['label'] == 1])

In [ ]:
results.iloc[0]

In [ ]:
result_df = pd.DataFrame(results)
result_df

In [ ]:
result_df.describe()

In [ ]:
result_df.groupby(["u", "v_pos", "edge_type", "ts"]).count()

In [ ]:
df.describe()

In [ ]:
df

In [ ]:
test_df = pd.merge(df[df["split"] == 2], result_df, on=["u", "v_pos", "edge_type", "ts", "v", "label"])
test_df

In [ ]:
test_df.groupby(["u", "v_pos", "edge_type", "ts"]).count()

In [ ]:
test = result_df.groupby(["u", "v_pos", "edge_type", "ts"]).count()
test[test["v"] > 1]

In [ ]:
from evaluation import evaluate
from collections import defaultdict

In [ ]:
# edge_groups = defaultdict(lambda: {'pos_score': None, 'neg_scores': []})
# for item in results:
#     # Use v_pos as the target node for grouping, which is the true target for all edges
#     v_for_grouping = item.get('v_pos', item['v'])
#     key = (item['u'], item.get('edge_type'), v_for_grouping, item['ts'])
#     score = item['score']  # Now using the adjusted score
#     length = item['length']

#     if item['label'] == 1:
#         edge_groups[key]['pos_score'] = (score, length)
#     else:
#         edge_groups[key]['neg_scores'].append((score, length))

In [ ]:
# final = evaluate(edge_groups, verbose=False)
final = evaluate(results, verbose=False)
final

In [ ]:
test_df

In [ ]:
def process(group):
    return group.loc[group["label"] == 0, "length"].min()

In [ ]:
test_df.groupby(["u", "v_pos", "edge_type", "ts"]).apply(process, include_groups=False)

In [ ]:
check = pd.merge(result_df, plot_df, on=["u", "v", "v_pos", "ts", "edge_type"])
check

In [ ]:
check_df = check[check['length'] != check['path_length']]
check_df

In [ ]:
check_df.describe()